# Libraries

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# Database

In [2]:
url = ['https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx']

In [3]:
online = pd.read_excel(url[0])

In [4]:
online.dropna(subset=['Customer ID'], inplace = True)

# Functions

<ul>
 <li>Function that takes a date as input and returns month format</li>
</ul>

In [5]:
def get_month(x):
    return dt.datetime(x.year, x.month, 1)

<ul>
 <li>Function that takes a date as input and returns year, month and firts day</li>
</ul>

In [6]:
def get_date_int(df, column):
    
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day

    return year, month, day

# Organizing Database

<ul>
 <li>Exploring the <i>Online</i> dataset.</i></li>
</ul>

In [7]:
online.sample(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
175862,506111,21866,UNION JACK FLAG LUGGAGE TAG,6,2010-04-27 13:44:00,1.25,12849.0,United Kingdom
183182,506820,21171,BATHROOM METAL SIGN,24,2010-05-04 12:49:00,1.45,17504.0,United Kingdom
396642,527399,22739,RIBBON REEL CHRISTMAS SOCK BAUBLE,10,2010-10-17 13:58:00,1.65,15590.0,United Kingdom
115756,500362,82580,BATHROOM METAL SIGN,4,2010-03-07 16:12:00,0.55,17449.0,United Kingdom
473093,533969,21207,SKULL AND CROSSBONES GARLAND,1,2010-11-19 13:16:00,1.65,18022.0,United Kingdom


<ul>
 <li><i>Online</i> dataset has an identifier for each order, however, <i>Cancelled Orders</i> starts with letter C, they are going to be removed.</li>
</ul>

In [8]:
online = online.loc[~online.Invoice.str.contains('C', na = False)]

<ul>
 <li>In order to calculate the <i>Recency</i> metric, a <i>today_date</i> variable will be defined taking into account the <i>Invoice Date</i> range.</li>
</ul>

In [9]:
print(f'Max Invoice date {online.InvoiceDate.max()}',f'\nMin Invoice date {online.InvoiceDate.min()}')

Max Invoice date 2010-12-09 20:01:00 
Min Invoice date 2009-12-01 07:45:00


In [10]:
today_date = pd.to_datetime("2011-12-11")
today_date

Timestamp('2011-12-11 00:00:00')

<ul>
 <li>In order to calculate the <i>Monetary Value</i> metric, <i>TotalSum</i> column is calculated as the result of <i>Quantity</i> and <i>Price.</i></li>
</ul>

In [11]:
online['TotalSum'] = online.Price * online.Quantity

# RFM metrics

<ul>
 <li>Calculating RFM metrics for each <i>Customer</i>.</li>
</ul>

In [49]:
grouping = online.groupby('Customer ID')

rfm = grouping.agg({'InvoiceDate':lambda x: (today_date - x.max()).days,
                    'Invoice': lambda x: x.nunique(),
                    'TotalSum': lambda x: x.sum()})

rfm.columns = ['Recency', 'Frequency', 'Monetary'] 
rfm.sort_values(by = ['Frequency'],ascending=True, inplace= True)

print(rfm.shape)
rfm.head(10)

(4314, 3)


,Recency,Frequency,Monetary
Customer ID,,,
16178.0,402,1,414.70
14271.0,410,1,106.81
14269.0,440,1,295.73
14268.0,471,1,354.85
14266.0,429,1,40.56
14263.0,424,1,322.93
16193.0,411,1,315.98
17369.0,456,1,979.20
15851.0,527,1,109.73


# Scoring RFM Metrics

<ul>
 <li><i>Recency</i> metric by <i>Customer ID.</i></li>
</ul>

In [51]:
r_labels = range(4,0,-1)
r_quartiles = pd.qcut( rfm['Recency'], 4, labels = r_labels)
rfm = rfm.assign( R = r_quartiles.values)
rfm.head(10)

,Recency,Frequency,Monetary,R
Customer ID,,,,
16178.0,402,1,414.70,3
14271.0,410,1,106.81,3
14269.0,440,1,295.73,2
14268.0,471,1,354.85,2
14266.0,429,1,40.56,2
14263.0,424,1,322.93,2
16193.0,411,1,315.98,3
17369.0,456,1,979.20,2
15851.0,527,1,109.73,1


<ul>
 <li><i>Frequency</i> metric by <i>Customer ID.</i></li>
</ul>

In [54]:
f_labels = range(1,5)
f_quartiles = rfm['Frequency'].transform(lambda x: pd.qcut(x.rank(method = 'first'), q = 4, labels = f_labels))
rfm = rfm.assign( F = f_quartiles.values)
rfm.sample(10)

,Recency,Frequency,Monetary,R,F
Customer ID,,,,,
12413.0,403,1,241.35,3,2
15257.0,415,1,368.75,3,1
16350.0,375,5,1137.42,4,4
12779.0,372,7,2740.91,4,4
14579.0,433,1,105.56,2,1
15812.0,436,7,3400.33,2,4
17003.0,403,4,830.16,3,3
14970.0,410,8,3327.29,3,4
16294.0,477,1,126.05,2,2


<ul>
 <li><i>Monetary</i> metric by <i>Customer ID.</i></li>
</ul>

In [55]:
m_labels = range(1,5)
m_quartiles = rfm['Monetary'].transform(lambda x: pd.qcut(x.rank(method = 'first'), q = 4, labels = f_labels))
rfm = rfm.assign( M = m_quartiles.values)
rfm.sample(10)

,Recency,Frequency,Monetary,R,F,M
Customer ID,,,,,,
13405.0,482,3,641.36,2,3,2
13129.0,453,2,233.25,2,2,1
17821.0,407,1,334.62,3,2,2
17323.0,408,5,644.52,3,3,2
12490.0,379,11,4405.71,4,4,4
14042.0,726,1,431.98,1,1,2
17473.0,409,2,607.00,3,2,2
17403.0,451,1,118.95,2,1,1
13304.0,456,3,917.57,2,3,3


# RFM segment and RFM score

<ul>
 <li>Calculating <i>RFM segment</i> and <i>RFM score</i> based on each metric.</li>
</ul>

In [ ]:
def join_rfm(x): 
    return str(x['R']) + str(x['F']) + str(x['M'])

In [57]:
rfm['RFM_Segment'] = rfm.apply(join_rfm, axis = 1)
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis = 1)

rfm.sample(10)

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score
Customer ID,,,,,,,,
17236.0,411,6,1013.13,3,4,3,343,10
13519.0,443,1,220.46,2,2,1,221,5
16519.0,366,3,341.18,4,3,2,432,9
16941.0,612,1,33.00,1,1,1,111,3
17371.0,624,3,238.66,1,3,1,131,5
16944.0,610,1,101.36,1,1,1,111,3
13320.0,435,2,679.40,2,2,2,222,6
17141.0,419,4,754.33,2,3,3,233,8
16908.0,723,1,252.59,1,1,1,111,3


# Analyzing RFM segments

<ul>
 <li>Number of <i>Customers</i> by <i>RFM segment.</i></li>
</ul>

In [60]:
rfm.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    463
111    373
344    221
333    169
233    160
433    149
211    144
121    143
122    131
112    130
dtype: int64

<ul>
 <li><i>RFM Segments</i> with their average metrics.</li>
</ul>

In [69]:
rfm.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean','count']}).round(1)

Recency Frequency Monetary      
             mean      mean     mean count
RFM_Score                                 
3           620.6       1.0    163.0   373
4           560.2       1.1    251.6   417
5           493.4       1.3    348.3   457
6           458.0       1.6    529.6   484
7           449.0       2.4    824.1   458
8           425.0       2.9   1090.5   456
9           412.4       4.0   1548.1   422
10          403.1       5.6   2389.6   409
11          387.7       8.4   3355.2   375
12          372.4      16.1   9691.5   463

# Grouping Into Named Segments

<ul>
 <li><i>RFM Segments</i> with their average metrics.</li>
</ul>

In [70]:
def segment_me(df):
    if df['RFM_Score'] >= 9:
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'

In [75]:
rfm['General_Segment'] = rfm.apply(segment_me, axis=1)

rfm.groupby('General_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean','count']}).round(1)

Recency Frequency Monetary      
                   mean      mean     mean count
General_Segment                                 
Bronze            588.7       1.1    209.8   790
Gold              393.5       8.7   4419.4  1669
Silver            456.4       2.0    695.5  1855